In [ ]:
#import the necessary modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

In [ ]:
#print the columns with missing values and number of them

def missing_values(data):
    col_missing = 0
    
    print('COLUMNS WITH MISSING VALUES: \n')

    for col in data.columns: #check each column of the training dataset
        if data[col].isnull().sum() != 0: #if there are missing values in that column
            print(f'{col}, {data[col].isnull().sum()}') #print the column name a                                                                         number of missing values
            col_missing += 1

    if col_missing == 0:
        print('There are no columns with missing values!')

In [ ]:
#some columns with missing values need to be fixed, because Nan indicates lack of a service instead of lack of the data

def fix_missing_values(data):
    data['Alley'].fillna('NoAlleyAccess', inplace = True)

    data['BsmtQual'].fillna('NoBasement', inplace = True)
    data['BsmtCond'].fillna('NoBasement', inplace = True)
    data['BsmtExposure'].fillna('NoBasement', inplace = True)
    data['BsmtFinType1'].fillna('NoBasement', inplace = True)
    data['BsmtFinType2'].fillna('NoBasement', inplace = True)

    data['FireplaceQu'].fillna('NoFireplace', inplace = True)

    data['GarageType'].fillna('NoGarage', inplace = True)
    data['GarageFinish'].fillna('NoGarage', inplace = True)
    data['GarageQual'].fillna('NoGarage', inplace = True)
    data['GarageCond'].fillna('NoGarage', inplace = True)

    data['PoolQC'].fillna('NoPool', inplace = True)

    data['Fence'].fillna('NoFence', inplace = True)

    data['MiscFeature'].fillna('None', inplace = True)

In [ ]:
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

# Load the data

In [ ]:
#load the training and testing datasets
training_data = pd.read_csv('train.csv')
testing_data = pd.read_csv('test.csv')

#target variable
target = training_data['SalePrice']

#drop 'SalePrice' column in the training dataset
training_data.drop(columns = 'SalePrice', inplace = True)

#merge the training and testing datasets
#data_merged = pd.concat([training_data, testing_data], axis = 0)

In [ ]:
print(training_data.shape)
print(testing_data.shape)
#print(data_merged.shape)

In [ ]:
training_data.head()

# Treat the missing values

In [ ]:
#check for missing values
print('TRAINING DATASET')
missing_values(training_data)

print('\n\n')

print('TESTING DATASET')
missing_values(testing_data)

In [ ]:
#fix the missing values that indicate a lack of feature and not a data not available
fix_missing_values(training_data)
fix_missing_values(testing_data)

#check for missing values
print('TRAINING DATASET')
missing_values(training_data)

print('\n\n')

print('TESTING DATASET')
missing_values(testing_data)

In [ ]:
#Garage built more in the past are in worse condition
sns.boxplot(training_data['GarageCond'], training_data['GarageYrBlt'])

In [ ]:
#remove 'GarageYrBlt' column, because if there is no garage no built year is available. In addition, we use the fact that a newer garage is reflected in a better condition. 
if 'GarageYrBlt' in training_data.columns:
    training_data.drop(columns = 'GarageYrBlt', inplace = True)
else:
    print('\'GarageYrBlt\' column already removed in training dataset')

if 'GarageYrBlt' in testing_data.columns:
    testing_data.drop(columns = 'GarageYrBlt', inplace = True)
else:
    print('\'GarageYrBlt\' column already removed in testing dataset')

In [ ]:
training_data.corrwith(training_data['LotFrontage']).sort_values(ascending = False)[0:5]

In [ ]:
#linear regression between 'LotAreaSqrt' and 'LotFrontage'
training_data['LotAreaSqrt'] = np.sqrt(training_data['LotArea'])
X_lf = sm.add_constant(training_data[training_data['LotFrontage'].notnull()]['LotAreaSqrt'])
y_lf = training_data['LotFrontage'].dropna()
model_lf = sm.OLS(y_lf, X_lf)
results_lf = model_lf.fit()
#print(results_lf.summary())

print(f'Parameters: {results_lf.params}')
print(f'R2: {results_lf.rsquared}')

#plot the results of the linear regression
plt.scatter(training_data['LotAreaSqrt'], training_data['LotFrontage'])
plt.plot(training_data['LotAreaSqrt'], results_lf.params[0] + results_lf.params[1] * training_data['LotAreaSqrt'], '-r')
plt.xlabel('LotAreaSqrt')
plt.ylabel('LotFrontage')

training_data.drop(columns = 'LotAreaSqrt', inplace = True)

In [ ]:
training_data.corrwith(training_data['LotFrontage']).sort_values(ascending = False)[0:5]

#'LotFrontage' and 'LotAreaSqrt' are correlated, so I can use this relationship to replace the missing values in 'LotFrontage'

In [ ]:
#Replace missing values of 'LotFrontage' with square root of 'LotArea' (see linear regression above)
training_data['LotFrontage'].fillna(results_lf.params[0] + results_lf.params[1] * np.sqrt(training_data['LotArea']), inplace = True)

testing_data['LotFrontage'].fillna(results_lf.params[0] + results_lf.params[1] * np.sqrt(testing_data['LotArea']), inplace = True)

In [ ]:
#check for missing values
print('TRAINING DATASET')
missing_values(training_data)

print('\n\n')

print('TESTING DATASET')
missing_values(testing_data)

In [ ]:
#Drop the rows with the remaining missing values
training_data.dropna(axis = 0, inplace = True)
testing_data.dropna(axis = 0, inplace = True)

print(f'Dimensions training dataset: {training_data.shape}')
print(f'Dimensions testing dataset: {testing_data.shape}')

In [ ]:
#check for missing values
print('TRAINING DATASET')
missing_values(training_data)

print('\n\n')

print('TESTING DATASET')
missing_values(testing_data)

# One-hot encoding of categorical variables

In [73]:
from sklearn.preprocessing import OneHotEncoder

#find categorical features
categorical_cols = training_data.columns[training_data.dtypes == 'object']
# creating one hot encoder object 
encoder = OneHotEncoder(drop = 'first')
#encode the categorical variable
X = encoder.fit_transform(training_data[categorical_cols])
#To add this back into the original dataframe 
df_encoded = pd.DataFrame(X.toarray(), columns = encoder.get_feature_names(categorical_cols))
#Concatenate training_data with encoded dataframe
training_data_encoded = pd.concat([training_data, df_encoded], axis=1)
#Drop the original categorical columns because they are encoded now
training_data_encoded.drop(columns = categorical_cols, inplace = True) 

In [74]:
training_data_encoded.shape

(1460, 259)